In [ ]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 4.6 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633721 sha256=fa67ca5a4d817a0fd89f03d08fe09fadd66c8c07d71b3b3586d24f8791c5b8e6
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
import pandas as pd

products = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CP1/products.csv')
up_reorder = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CP1/up_reorder.csv')

print('products shape :', products.shape)
print('up_reorder shape :', up_reorder.shape)

products shape : (49688, 4)
up_reorder shape : (5325258, 3)


In [ ]:
from surprise import Reader, Dataset, accuracy, SVD
from surprise.model_selection import cross_validate

reader = Reader(rating_scale=(2.0, 5.0))
data = Dataset.load_from_df(up_reorder[['user_id', 'product_id', 'up_reorder']], reader=reader)
# train = data.build_full_trainset()

algo = SVD(random_state=42)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9816  0.9837  0.9832  0.9823  0.9824  0.9827  0.0007  
MAE (testset)     0.8416  0.8431  0.8427  0.8424  0.8421  0.8424  0.0005  
Fit time          277.50  282.85  283.60  285.45  284.61  282.80  2.79    
Test time         19.32   18.31   17.03   18.13   17.92   18.14   0.73    


{'fit_time': (277.49904584884644,
  282.85254645347595,
  283.5954222679138,
  285.45129346847534,
  284.61088967323303),
 'test_mae': array([0.84164864, 0.843089  , 0.84267208, 0.84236137, 0.84206523]),
 'test_rmse': array([0.98163512, 0.9837319 , 0.98318793, 0.98232728, 0.98239607]),
 'test_time': (19.315064668655396,
  18.30753445625305,
  17.02649450302124,
  18.125556468963623,
  17.922212839126587)}

In [ ]:
import pickle

with open("algo.pickle","wb") as fw:
  pickle.dump(algo, fw)

In [ ]:
up_reorder = up_reorder[up_reorder['up_reorder']>=4].groupby('user_id')['product_id'].apply(list).reset_index()
up_reorder

user_id                                         product_id
0             1                         [196, 10258, 12427, 25133]
1             2  [1559, 2002, 12000, 16589, 18523, 19051, 19156...
2             3   [9387, 17668, 18599, 21903, 39190, 43961, 47766]
3             5                                     [11777, 26604]
4             7  [4920, 6361, 13198, 17638, 21137, 27690, 30391...
...         ...                                                ...
153470   206203                              [15693, 21469, 37188]
153471   206206  [11520, 11954, 13045, 23737, 27086, 38530, 401...
153472   206207  [2954, 3397, 3469, 11068, 11140, 13176, 19678,...
153473   206208  [365, 1463, 1804, 2643, 5876, 6297, 9339, 1152...
153474   206209  [6846, 9405, 13176, 19348, 23594, 24852, 38167...

[153475 rows x 2 columns]

In [ ]:
def sortkey_est(pred):
  return pred.est

def find_name(arg):
  return products[products['product_id']==arg]['product_name'].values[0]

def recommend(user_id):
  dic = {}
  try:
    a = up_reorder[up_reorder['user_id']==user_id]['product_id'].values[0]
  except:
    return 'No recommend'
  # k = 2
  for i in a:
    aisle = products[products['product_id']==i]['aisle_id'].values[0]
    department = products[products['product_id']==i]['department_id'].values[0]
    candi = products[(products['aisle_id']==aisle) & (products['department_id']==department)]['product_id'].to_list()
    predi = [algo.predict(str(user_id), str(j)) for j in candi]
    predi.sort(key=sortkey_est, reverse=True)
    top_predi = predi[:3]
    top_product_id = [int(pred.iid) for pred in top_predi]
    top_product_name = products[products.product_id.isin(top_product_id)]['product_name'].to_list()
    # if find_name(aisle)+'/'+find_name(department) in dic:
    #   dic[find_name(aisle)+'/'+find_name(department)+str(k)] = top_product_name
    #   k += 1
    # else:
    #   dic[find_name(aisle)+'/'+find_name(department)] = top_product_name
    dic[find_name(aisle)+'/'+find_name(department)] = top_product_name
  return dic

In [ ]:
recommend(1)

{'Coconut Chocolate Chip Energy Bar/Pure Coconut Water With Orange': ['Jasmine Tea Unfiltered Ginger Ale',
  'Sparkling Blush Grape Juice',
  'Soda'],
 'Organic Turkey Burgers/Gluten Free Quinoa Three Cheese & Mushroom Blend': ['Healthy Pop Butter Popcorn',
  'Uncured Cracked Pepper Beef',
  'Movie Theater Butter Pop Corn'],
 'Petit Suisse Fruit/Gluten Free Quinoa Three Cheese & Mushroom Blend': ['Dried Sweetened Cranberries',
  'Dry Roasted Pistachios',
  'Ground Flaxseed With Mixed Berries'],
 'Small & Medium Dental Dog Treats/Mint Chocolate Flavored Syrup': ['Sharp Cheddar',
  'Extra Sharp Cheddar Cheese',
  'Shaved Parmesan Cheese']}